## Columbia University
### ECBM E4040 Neural Networks and Deep Learning. Fall 2024.

## Assignment 2 - Task 1: Optimization

In this task, we introduce several improved optimization algorithms based on stochastic gradient descent (SGD). Naive SGD is a reasonable method to update neural network parameters. However, there exists two main drawbacks:

- First, to make SGD perform well, one would need to find an appropriate learning rate and good initial values for the prameters. The training will progress slowly if the learning rate is small, or diverge if the learning rate is too large. Since we often have no prior knowledge about the training data in reality, it is not trivial to find a good learning rate by hand. Also, when the network grows deeper, one may need to set a different learning rate for each layer. 

- Second, SGD strictly follows the gradients of the **batched data** when updating the parameters. This can be problematic with real-world problems as has been demonstrated in the lectures.

To seek for improvements of naive SGD, momentum, parameter estimation and adaptive learning rate methods are commonly the ones to rely on. Here, you are going to experiment with **SGD with Momentum**, **SGD with Nesterov-accelerated Momentum**, **Adam**, **AdamW**, **RMSProp** and compare their performances.

Consult the slides and [text book](https://www.deeplearningbook.org) for details. Here is also [a useful link](http://ruder.io/optimizing-gradient-descent/) to learn more about some methods used in this task.

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Import modules
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist

## Load Fashion-MNIST

Here we use a small dataset with only 2500 samples to simulate the "lack-of-data" situation.

In [5]:
# Load the raw Fashion-MNIST data.
train, val = fashion_mnist.load_data()

X_train_raw, y_train = train
X_val_raw, y_val = val

X_train = X_train_raw.reshape((X_train_raw.shape[0], X_train_raw.shape[1]**2))
X_val = X_val_raw.reshape((X_val_raw.shape[0], X_val_raw.shape[1]**2))

#Consider a subset of 2500 samples of the 60000 total images (indexed 10000 ~ 12500)
X_val = X_train[10000:10500,:]
y_val = y_train[10000:10500]
X_train = X_train[10500:12500,:]
y_train = y_train[10500:12500]

mean_image = np.mean(X_train, axis=0).astype(np.float32)
X_train = X_train.astype(np.float32) - mean_image
X_val = X_val.astype(np.float32) - mean_image

# We have vectorized the data for you
# Flatten the 32×32×3 images into 1×3072 Numpy arrays
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)

Training data shape:  (2000, 784)
Training labels shape:  (2000,)
Validation data shape:  (500, 784)
Validation labels shape:  (500,)


## Part 1: Implement Several Optimizers (16%)

We provide code snippets for testing your code implementations.

The best anticipated achievable accuracies are specific for each algorithm. You may use these accuracies to judge the correctness of your implementations.

In [8]:
from utils.neuralnets.mlp import MLP

### Basics

Assume that the goal is to optimize an objective function $L$ parametrized by network weights $\theta \in R^d$, the update rule of an iterative optimization algorithm in general can be formulated as

$$\theta_{t+1} \gets \theta_t + \alpha_t p_t$$

where $\alpha_t > 0$ is the **step size** and $p$ is the **direction of update**. 

Both $\alpha$ and $p$ can be proposed in numerous different ways which result in different optimizers with different performances.

Note that in the following equations, we ***DO NOT*** take learning rate decay into consideration. This has been implemented in the base class `Optimizer.train()`. All optimizers will be derived from this class.

### Original SGD (For Comparison)

At time step $t$, let the gradient of a real-valued loss function $L$ w.r.t network parameter $\theta$ be given by

$$g_t = \nabla_{\theta_t} L(\theta_t)$$

and $\theta_t$ denotes the values of the parameters at time $t$.

As you have seen in previous examples, the loss $L$ is calculated from a mini-batch stochastically sampled from the entire dataset.

SGD (Stochastic Gradient Descent) algorithm is formulated as

$$\theta_{t+1} = \theta_t - \eta g_t$$

where $\eta$ is the ***learning rate***. 

The final accuracy you should expect is arround 0.1-0.3. 

In [13]:
from utils.optimizers import SGDOptim

model = MLP(
    input_dim=X_train.shape[1], hidden_dims=[100, 100],
    num_classes=10, weight_scale=1e-3, l2_reg=0.0
)
optimizer = SGDOptim(model)
hist_sgd = optimizer.train(
    X_train, y_train, X_val, y_val, 
    num_epoch=15, batch_size=200, learning_rate=1e-2, learning_decay=0.95, 
    verbose=False, record_interval=1
)

number of batches for training: 10
epoch 1: valid acc = 0.116, new learning rate = 0.0095, number of evaluations 12
epoch 2: valid acc = 0.192, new learning rate = 0.009025, number of evaluations 12
epoch 3: valid acc = 0.194, new learning rate = 0.00857375, number of evaluations 12
epoch 4: valid acc = 0.198, new learning rate = 0.0081450625, number of evaluations 12
epoch 5: valid acc = 0.208, new learning rate = 0.007737809374999999, number of evaluations 12
epoch 6: valid acc = 0.2, new learning rate = 0.007350918906249998, number of evaluations 12
epoch 7: valid acc = 0.202, new learning rate = 0.006983372960937498, number of evaluations 12
epoch 8: valid acc = 0.204, new learning rate = 0.006634204312890623, number of evaluations 12
epoch 9: valid acc = 0.202, new learning rate = 0.006302494097246091, number of evaluations 12
epoch 10: valid acc = 0.202, new learning rate = 0.005987369392383786, number of evaluations 12
epoch 11: valid acc = 0.204, new learning rate = 0.005688000

#### SGD + Momentum

All gradient methods share the drawback of potentially being stuck in local minima. Momentum is one possible solution to circumvent this problem. By accumulating the "velocities" of past updates, the algorithm may be able to escape local minima and give faster convergence.

The algorithm is formulated as two steps:
1. Accumulate the **velocities**:
   $$v_t = \beta v_{t-1} + g_t$$
2. Update the parameters:
   $$\theta_{t+1} = \theta_t - \eta v_t$$

where $\beta \in [0, 1]$ is the decay factor controlling the effect of past velocities on the current update and $v_0$ is initialized to be $\mathbb{0}$.

The intuition is to modify the current update direction (naively $g_t$ in SGD) by accumulated past directions. You can think of it as rolling a heavy ball down the hill, where the inertia of the ball will always try to retain the direction of the current velocity instead of simply following the slope.

The final accuracy you should expect is arround 0.5-0.7. 

<span style="color:red">__TODO:__</span> Implement SGD + Momentum by completing `SGDmomentumOptim` in **./utils/optimizers.py**

In [47]:
# Verification code for your implemention
# Please don't change anything.

from utils.optimizers import SGDMomentumOptim

model = MLP(
    input_dim=X_train.shape[1], hidden_dims=[100, 100],
    num_classes=10, weight_scale=1e-3, l2_reg=0.0
)
optimizer = SGDMomentumOptim(model, momentum=0.8)
hist_sgd_momentum = optimizer.train(
    X_train, y_train, X_val, y_val, 
    num_epoch=15, batch_size=200, learning_rate=1e-2, 
    learning_decay=0.95, verbose=False, record_interval=1
)

number of batches for training: 10
epoch 1: valid acc = 0.152, new learning rate = 0.0095, number of evaluations 12
epoch 2: valid acc = 0.198, new learning rate = 0.009025, number of evaluations 12
epoch 3: valid acc = 0.204, new learning rate = 0.00857375, number of evaluations 12
epoch 4: valid acc = 0.218, new learning rate = 0.0081450625, number of evaluations 12
epoch 5: valid acc = 0.224, new learning rate = 0.007737809374999999, number of evaluations 12
epoch 6: valid acc = 0.214, new learning rate = 0.007350918906249998, number of evaluations 12
epoch 7: valid acc = 0.21, new learning rate = 0.006983372960937498, number of evaluations 12
epoch 8: valid acc = 0.336, new learning rate = 0.006634204312890623, number of evaluations 12
epoch 9: valid acc = 0.404, new learning rate = 0.006302494097246091, number of evaluations 12
epoch 10: valid acc = 0.418, new learning rate = 0.005987369392383786, number of evaluations 12
epoch 11: valid acc = 0.422, new learning rate = 0.00568800

#### SGD + Nesterov-accelerated Momentum

While momentum shows exciting results for such a simple algorithm, the convergence rate can be further improved by introducing the Nesterov acceleration.

As an intuition, the previous algorithm only modify the parameters using the momentum at the **current** time step $v_t$. Nesterov proposed to derive an estimation of the momentum at the **next** time step $\hat v_{t+1}$ to perform the update.

This translates to the following algorithm:
1. Accumulate the velocities:
   $$v_t = \beta v_{t-1} + g_t$$
2. **Nesterov acceleration**:
   $$\hat v_{t+1} = \beta v_t + g_t$$
3. Update the parameters:
   $$\theta_{t+1} = \theta_t - \eta \hat v_{t+1}$$

The final accuracy you should expect is arround 0.7-0.9. 

<span style="color:red">__TODO:__</span> Implement SGD + Nesterov-accelerated Momentum by completing `SGDNestMomentumOptim` in **./utils/optimizers.py**

In [33]:
# Verification code for your implemention
# Please don't change anything.

from utils.optimizers import SGDNestMomentumOptim

model = MLP(
    input_dim=X_train.shape[1], hidden_dims=[100, 100],
    num_classes=10, weight_scale=1e-3, l2_reg=0.0
)
optimizer = SGDNestMomentumOptim(model, momentum=0.9)
hist_sgd_nesterov = optimizer.train(
    X_train, y_train, X_val, y_val, 
    num_epoch=15, batch_size=200, learning_rate=1e-2, 
    learning_decay=0.95, verbose=False, record_interval=1
)

number of batches for training: 10
epoch 1: valid acc = 0.2, new learning rate = 0.0095, number of evaluations 12
epoch 2: valid acc = 0.228, new learning rate = 0.009025, number of evaluations 12
epoch 3: valid acc = 0.204, new learning rate = 0.00857375, number of evaluations 12
epoch 4: valid acc = 0.276, new learning rate = 0.0081450625, number of evaluations 12
epoch 5: valid acc = 0.32, new learning rate = 0.007737809374999999, number of evaluations 12
epoch 6: valid acc = 0.422, new learning rate = 0.007350918906249998, number of evaluations 12
epoch 7: valid acc = 0.39, new learning rate = 0.006983372960937498, number of evaluations 12
epoch 8: valid acc = 0.482, new learning rate = 0.006634204312890623, number of evaluations 12
epoch 9: valid acc = 0.626, new learning rate = 0.006302494097246091, number of evaluations 12
epoch 10: valid acc = 0.6, new learning rate = 0.005987369392383786, number of evaluations 12
epoch 11: valid acc = 0.706, new learning rate = 0.0056880009227

#### Adam

With the algorithm above, we're already implying the notion of **[moment](https://en.wikipedia.org/wiki/Moment_(mathematics)) estimation** (i.e. $\hat v_{t+1}$) in the sense of Nesterov by combining current gradient and exponetially decaying past gradients (with factor $\beta$). Here we explore another more flexible and widely used estimation method - the **moving average**.

For any time series denoted by $x_0, x_1, \dots, x_t$, its moving average at time $t$ is defined iteratively as a weighted summation of the previous average and current value, i.e.

$$\mu_t = \beta \mu_{t-1} + (1 - \beta) x_t$$

where the hyperparameter $\beta \in [0, 1]$ is the decay factor controlling the influence of the past to the present. This provides the algorithm with better adaptations to the change of parameters through time.

Naturally, we would want to know *statistically* how far our moving average $\mu_t$ deviates from the true average $\mu$ by considering its expectation $\mathbb{E}[\mu_t]$.

Specifically, the moving average can be expanded (by plugging in each $\mu_t$) to

$$\mu_t = (1 - \beta) \sum_{i=1}^t \beta^{t-i} x_i$$

Then

$$
\mathbb{E}[\mu_t]
= \mathbb{E} \left[ (1 - \beta) \sum_{i=1}^t \beta^{t-i} x_i \right]
= (1 - \beta) \sum_{i=1}^t \beta^{t-i} \mathbb{E}[x_i]
$$

Assume that the series elements are identically distributed (i.e. $\mathbb{E}[x_1] = \dots = \mathbb{E}[x_t] = \mu$), then

$$\mathbb{E}[\mu_t] = \mu (1 - \beta) \sum_{i=1}^t \beta^{t-i} = (1 - \beta^t) \mu$$

Therefore, a bias-corrected moving average can be given by

$$\hat \mu_t = \frac{\mu_t}{1 - \beta^t}$$

With all the discussions above, Adam (Adaptive Moment Estimation) algorithm can be formulated into three following steps.

**1. Moment calculation**
   - The 1st moment (**mean**) of the gradients:
     $$m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t$$
   - The 2nd moment (**variance**) of the gradients:
     $$v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2$$

**2. Bias correction**

   - Adjust the 1st moment:
   $$\hat m_t = \frac{m_t}{1 - \beta_1^t}$$
   - Adjust the 2nd moment:
   $$\hat v_t = \frac{v_t}{1 - \beta_2^t}$$

**3. Parameter update**

   $$\theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{\hat v_t}+\epsilon} \odot \hat m_t$$

   Here, $\epsilon$ is a small value (e.g. 1e-8) serves to avoid zero-division and $\odot$ denotes the Hadamard (element-wise) product.

The final accuracy you should expect is arround 0.7-0.9. 

**This is usually (not always) the optimal choice in practice.**

<span style="color:red">__TODO:__</span> Implement Adam by editing `AdamOptim` in **./utils/optimizers.py**

In [50]:
# Verification code for your implemention
# Please don't change anything.

from utils.optimizers import AdamOptim

model = MLP(
    input_dim=X_train.shape[1], hidden_dims=[100, 100],
    num_classes=10, weight_scale=1e-3, l2_reg=0.0
)
optimizer = AdamOptim(model, beta1=0.9, beta2=0.99)
hist_adam = optimizer.train(
    X_train, y_train, X_val, y_val, 
    num_epoch=15, batch_size=200, learning_rate=1e-3, 
    learning_decay=0.95, verbose=False, record_interval=1
)

number of batches for training: 10
epoch 1: valid acc = 0.218, new learning rate = 0.00095, number of evaluations 12
epoch 2: valid acc = 0.224, new learning rate = 0.0009025, number of evaluations 12
epoch 3: valid acc = 0.202, new learning rate = 0.000857375, number of evaluations 12
epoch 4: valid acc = 0.332, new learning rate = 0.0008145062499999999, number of evaluations 12
epoch 5: valid acc = 0.39, new learning rate = 0.0007737809374999998, number of evaluations 12
epoch 6: valid acc = 0.48, new learning rate = 0.0007350918906249997, number of evaluations 12
epoch 7: valid acc = 0.484, new learning rate = 0.0006983372960937497, number of evaluations 12
epoch 8: valid acc = 0.59, new learning rate = 0.0006634204312890621, number of evaluations 12
epoch 9: valid acc = 0.634, new learning rate = 0.000630249409724609, number of evaluations 12
epoch 10: valid acc = 0.652, new learning rate = 0.0005987369392383785, number of evaluations 12
epoch 11: valid acc = 0.722, new learning ra

#### AdamW  (Adam with Weight Decay)

**AdamW** is a variant of the **Adam** optimizer that decouples the **weight decay** term from the gradient-based updates, making it more effective for improving generalization. Unlike the traditional Adam optimizer, which adds the weight decay term to the gradient update, AdamW explicitly separates these two aspects.

The update rule for AdamW is formulated as follows:

1. **Compute biased first moment (mean of gradients)**:
   $$
   m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t
   $$
   where $m_t$ represents the moving average of the gradient, $\beta_1$ is the decay rate for the first moment, and $g_t$ is the gradient at time step $t$.

2. **Compute biased second moment (uncentered variance of gradients)**:
   $$
   v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2
   $$
   where $v_t$ represents the moving average of the squared gradient, and $\beta_2$ is the decay rate for the second moment.

3. **Bias correction for moments**:
   - **First moment (bias-corrected)**:
   $$
   \hat{m}_t = \frac{m_t}{1 - \beta_1^t}
   $$
   - **Second moment (bias-corrected)**:
   $$
   \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
   $$

4. **Parameter update with weight decay**:
   $$
   \theta_{t+1} = \theta_t - \eta \left( \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} + \lambda \theta_t \right)
   $$
   where:
   - $\eta$ is the **learning rate**.
   - $\epsilon$ is a small constant (e.g., $10^{-8}$) to prevent division by zero.
   - $\lambda$ is the **weight decay** coefficient.

The final accuracy you should expect is arround **0.7-0.9**.


<span style="color:red">__TODO:__</span> Implement AdamW by editing `AdamWOptim` in **./utils/optimizers.py**

In [ ]:
# Verification code for your implemention
# Please don't change anything.

from utils.optimizers import AdamWOptim

model = MLP(
    input_dim=X_train.shape[1], hidden_dims=[100, 100],
    num_classes=10, weight_scale=1e-3, l2_reg=0.0
)
optimizer = AdamWOptim(model, beta1=0.9, beta2=0.999, eps=1e-8, l2_coeff = 1)
hist_adamw = optimizer.train(X_train, y_train, X_val, y_val, 
                                         num_epoch=15, batch_size=200, learning_rate=1e-3, 
                                         learning_decay=0.95, verbose=False, record_interval=1)



### RMSprop (Root Mean Square Propagation)

**RMSprop** is an adaptive learning rate optimization algorithm designed to deal with the challenges of adjusting learning rates, especially in scenarios where gradients can vary widely. RMSprop aims to resolve issues like oscillations and slow convergence by normalizing the learning rate based on recent gradients.

The update rule for **RMSprop** is formulated as follows:

1. **Compute the moving average of squared gradients**:
   $$
   v_t = \beta v_{t-1} + (1 - \beta) g_t^2
   $$
   where $v_t$ represents the moving average of the squared gradients, $\beta$ is a hyperparameter that controls the decay rate, and $g_t$ is the gradient at time step $t$.

2. **Update the parameters**:
   $$
   \theta_{t+1} = \theta_t - \frac{\eta}{\sqrt{v_t} + \epsilon} g_t
   $$
   where $\eta$ is the **learning rate**, and $\epsilon$ is a small value (e.g., $10^{-8}$) added to prevent division by zero.

The final accuracy you should expect is arround **0.7-0.9**.

<span style="color:red">__TODO:__</span> Implement RMSProp by editing `RMSpropOptim` in **./utils/optimizers.py**

In [ ]:
from utils.optimizers import RMSpropOptim

model = MLP(
    input_dim=X_train.shape[1], hidden_dims=[100, 100],
    num_classes=10, weight_scale=1e-3, l2_reg=0.0
)
optimizer = RMSpropOptim(model, gamma=0.9, eps=1e-12)
hist_rms = optimizer.train(X_train, y_train, X_val, y_val, 
                                         num_epoch=15, batch_size=200, learning_rate=1e-3, 
                                         learning_decay=0.95, verbose=False, record_interval=1)

## Part 2: Comparison (4%)

<span style="color:red">__TODO:__</span> Run the following cells, which plot the loss, training accuracy, and validation accuracy curves of different optimizers.

In [ ]:
loss_hist_sgd, train_acc_hist_sgd, val_acc_hist_sgd = hist_sgd
loss_hist_momentum, train_acc_hist_momentum, val_acc_hist_momentum = hist_sgd_momentum
loss_hist_nesterov, train_acc_hist_nesterov, val_acc_hist_nesterov = hist_sgd_nesterov
loss_hist_adam, train_acc_hist_adam, val_acc_hist_adam = hist_adam
loss_hist_adamw, train_acc_hist_adamw, val_acc_hist_adamw = hist_adamw
loss_hist_rms, train_acc_hist_rms, val_acc_hist_rms = hist_rms

In [ ]:
# Plot the training error curves for implemented optimizers
plt.plot(loss_hist_sgd, label="sgd")
plt.plot(loss_hist_momentum, label="momentum")
plt.plot(loss_hist_nesterov, label="nesterov")
plt.plot(loss_hist_adam, label="adam")
plt.plot(loss_hist_adamw, label="adamw")
plt.plot(loss_hist_rms, label="rms")
plt.legend()
plt.show()

In [ ]:
# Plot the training accuracy curves for implemented optimizers
plt.plot(train_acc_hist_sgd, label="sgd")
plt.plot(train_acc_hist_momentum, label="momentum")
plt.plot(train_acc_hist_nesterov, label="nesterov")
plt.plot(train_acc_hist_adam, label="adam")
plt.plot(train_acc_hist_adamw, label="adamw")
plt.plot(train_acc_hist_rms, label="rms")
plt.legend()
plt.show()

In [ ]:
# Plot the validation accuracy curves for implemented optimizers
plt.plot(val_acc_hist_sgd, label="sgd")
plt.plot(val_acc_hist_momentum, label="momentum")
plt.plot(val_acc_hist_nesterov, label="nesterov")
plt.plot(val_acc_hist_adam, label="adam")
plt.plot(val_acc_hist_adamw, label="adamw")
plt.plot(val_acc_hist_rms, label="rms")
plt.legend()
plt.show()

<span style="color:red">**TODO:**</span> Compare the results from above. Answer the following questions based on your observations and understandings of these optimizers:

1. Briefly conclude why each of the optimizers beats naive SGD. Which of them is the winner based on your results?
2. Discuss the trade-offs between using a simple optimizer like SGD versus a more complex one like Adam. In what scenarios might you prefer one over the other?
3. Briefly describe another optimizer that is not introduced in this assignment.

Answer: **[fill in here]**.